In [1]:
from ast import arg
import torch
import os
import time
from models.neural_network import *
import numpy as np
from torch.utils.data import Dataset,DataLoader,SubsetRandomSampler
import math
import argparse
from glob import glob
import pandas as pd
from tqdm import tqdm
from utils import *
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.io.arff import loadarff
from Dataset import *

device = torch.cuda.set_device(0)


/home/infres/ext-6343/venv_boverfitting/lib64/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:

data = loadarff('/home/infres/ext-6343/venv_boverfitting/benign-overfitting/data/BNG_wine_quality.arff')
raw_dataset = pd.DataFrame(data[0])
dataset = raw_dataset.copy()
dataset=dataset.sample(frac=1)[0:10000]

dataset = dataset.dropna()
dataset=(dataset-dataset.mean())/dataset.std()
dataset=dataset.iloc[0:380,:]
train_features = dataset.copy()
train_labels = train_features.pop('quality')

In [11]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                        na_values='?', comment='\t',
                        sep=' ', skipinitialspace=True) 
dataset = raw_dataset.copy()
dataset = dataset.dropna()
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')

dataset=(dataset-dataset.mean())/dataset.std()

train_features = dataset.copy()
train_labels = train_features.pop('MPG')
# train_features=(train_features-train_features.mean())/train_features.std()


In [35]:
X_train, X_test, y_train, y_test = train_test_split(train_features,train_labels)

X_train_list = torch.Tensor(X_train.values.astype(np.float32))
X_train_list=X_train_list.to(device)

y_train_list = torch.Tensor(y_train.values.astype(np.float32)).view(y_train.shape)
y_train_list=y_train_list.to(device)

X_test_list = torch.Tensor(X_test.values.astype(np.float32))
X_test_list=X_test_list.to(device)

y_test_list = torch.Tensor(y_test.values.astype(np.float32)).view(y_test.shape)
y_test_list=y_test_list.to(device)

In [36]:
model=NN(11,1,3000)
model.to(device)

fnet, params ,buffers =make_functional_with_buffers(model)


In [37]:
torch.cuda.empty_cache()
loss_fn = torch.nn.MSELoss() 
try:
    test_kernel_predicted=compute_kxkny(X_train_list,X_test_list,y_train_list,params,buffers,fnet)
    test_kernel_loss=loss_fn(test_kernel_predicted.reshape(y_test_list.shape[0]), y_test_list)
    print(test_kernel_loss.item())
except Exception as e:
    print(e)

1.0122417211532593


In [39]:
test_kernel_predicted

tensor([[-1.2018],
        [ 1.0171],
        [ 0.1097],
        [-0.8421],
        [-0.4836],
        [-1.0002],
        [-0.9008],
        [-0.5821],
        [-0.1199],
        [ 0.7260],
        [-0.0412],
        [-1.1562],
        [-0.6832],
        [ 0.6050],
        [ 0.8293],
        [ 0.0574],
        [-0.9808],
        [-0.4882],
        [-0.8950],
        [ 0.6186],
        [ 0.6180],
        [-0.2728],
        [-0.2877],
        [ 0.4857],
        [ 0.5935],
        [ 0.7342],
        [ 0.4671],
        [ 1.3508],
        [ 0.1545],
        [-0.2894],
        [ 0.4507],
        [-1.0716],
        [-0.0062],
        [ 0.6746],
        [ 0.0541],
        [-0.1667],
        [ 0.0502],
        [-0.2523],
        [-0.6103],
        [ 0.0806],
        [-0.3968],
        [ 0.7478],
        [-0.1237],
        [-1.3018],
        [ 1.0547],
        [-0.0735],
        [-0.2209],
        [-0.1303],
        [-0.1956],
        [ 0.6249],
        [ 1.1650],
        [ 0.5745],
        [-0.

In [27]:
y_test_list

tensor([ 1.3526, -0.9354, -0.9354,  1.3526,  0.2086,  0.2086,  1.3526,  0.2086,
         1.3526, -0.9354, -1.8838,  0.2086, -0.9354, -0.9354,  0.2086,  0.2086,
        -0.9354,  0.2086, -0.9354, -2.1369, -0.9354,  0.2086,  0.2086,  0.2086,
        -2.0424,  1.3526, -0.9354,  1.3526,  1.3526, -0.9354,  0.2086, -0.9354,
         1.3526,  0.2086, -0.9354,  0.2086, -0.9354, -0.9354, -0.9354,  0.2086,
        -0.9354,  0.2086,  0.2086,  0.2086,  0.2086,  1.3526, -0.9354,  0.2086,
        -0.9354,  0.2086,  0.2086,  0.2086, -0.9354,  1.3526, -0.9354, -0.9354,
         1.3526,  0.2086, -0.9354, -2.8046,  0.2086,  0.2086,  0.2086, -2.8636,
         0.2086,  0.2086,  0.2086,  0.2086,  0.2086,  1.3526,  1.3526,  0.2086,
         0.2086,  0.2086, -0.9354,  0.2086,  1.3526,  0.2086,  1.3526, -2.0275,
        -0.9354, -0.9354,  0.2086,  0.2086,  2.4373, -0.9354,  1.3526, -0.9354,
        -0.9354,  1.3526,  0.2086, -0.9354,  0.2086, -0.9354,  0.2086])